In [22]:
import pandas as pd 
import numpy as np 
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import Row 
from pyspark.sql.functions import avg 
import os 
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import col 
#from pyspark.sql.functions import round

In [2]:
os.environ["PYSPARK_PYTHON"] = "python"

In [3]:
df_pandas = pd.read_csv('seattle.csv')

# Some initial statisctics about dataset

In [4]:
df_pandas.head()

,Race/Ethnicity,Sex,Department,Age,Hourly Rate,Regular/Temporary,Employee Status
0,Hispanic or Latino,M,Parks & Recreation,42.0,36.57,R,A
1,Hispanic or Latino,F,Municipal Court,37.0,30.48,R,A
2,Hispanic or Latino,M,Police Department,46.0,57.70,R,A
3,Hispanic or Latino,F,Office of Labor Standards,NaN,46.68,R,A
4,Hispanic or Latino,M,Seattle Dept of Transportation,26.0,45.81,R,A


In [5]:
df_pandas.describe()

,Age,Hourly Rate
count,13652.000000,13659.000000
mean,44.665470,45.864420
std,13.400901,17.627389
min,15.000000,5.530000
25%,35.000000,32.850000
50%,45.000000,45.000000
75%,55.000000,57.870000
max,91.000000,187.767000


In [6]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13659 entries, 0 to 13658
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Race/Ethnicity     13659 non-null  object 
 1   Sex                13659 non-null  object 
 2   Department         13659 non-null  object 
 3   Age                13652 non-null  float64
 4   Hourly Rate        13659 non-null  float64
 5   Regular/Temporary  13659 non-null  object 
 6   Employee Status    13659 non-null  object 
dtypes: float64(2), object(5)
memory usage: 747.1+ KB


In [7]:
df_pandas.isna().sum()

Race/Ethnicity       0
Sex                  0
Department           0
Age                  7
Hourly Rate          0
Regular/Temporary    0
Employee Status      0
dtype: int64

In [45]:
df_pandas['Hourly Rate'].describe()

count    13659.000000
mean        45.864420
std         17.627389
min          5.530000
25%         32.850000
50%         45.000000
75%         57.870000
max        187.767000
Name: Hourly Rate, dtype: float64

In [46]:
df_pandas['Age'].describe()

count    13652.000000
mean        44.665470
std         13.400901
min         15.000000
25%         35.000000
50%         45.000000
75%         55.000000
max         91.000000
Name: Age, dtype: float64

In [52]:
df_pandas['Age'].value_counts().head(10)

51.0    393
52.0    386
41.0    354
38.0    353
53.0    351
37.0    348
34.0    341
50.0    339
55.0    338
40.0    336
Name: Age, dtype: int64

In [53]:
df_pandas['Age'].value_counts().tail(10)

78.0    9
79.0    6
86.0    4
85.0    2
82.0    2
83.0    1
81.0    1
84.0    1
91.0    1
15.0    1
Name: Age, dtype: int64

In [8]:
spark = SparkSession.builder.appName('Seattle_csv_spark').getOrCreate()

In [9]:
sparkDF=spark.createDataFrame(df_pandas) 
sparkDF.printSchema()
sparkDF.show()


root
 |-- Race/Ethnicity: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Hourly Rate: double (nullable = true)
 |-- Regular/Temporary: string (nullable = true)
 |-- Employee Status: string (nullable = true)

+------------------+---+--------------------+----+-----------+-----------------+---------------+
|    Race/Ethnicity|Sex|          Department| Age|Hourly Rate|Regular/Temporary|Employee Status|
+------------------+---+--------------------+----+-----------+-----------------+---------------+
|Hispanic or Latino|  M|  Parks & Recreation|42.0|      36.57|                R|              A|
|Hispanic or Latino|  F|     Municipal Court|37.0|      30.48|                R|              A|
|Hispanic or Latino|  M|   Police Department|46.0|       57.7|                R|              A|
|Hispanic or Latino|  F|Office of Labor S...| NaN|      46.68|                R|              A|
|Hispanic or Latin

In [10]:
sparkDF = sparkDF.replace(float('NaN'), None)

In [11]:
sparkDF.show()

+------------------+---+--------------------+----+-----------+-----------------+---------------+
|    Race/Ethnicity|Sex|          Department| Age|Hourly Rate|Regular/Temporary|Employee Status|
+------------------+---+--------------------+----+-----------+-----------------+---------------+
|Hispanic or Latino|  M|  Parks & Recreation|42.0|      36.57|                R|              A|
|Hispanic or Latino|  F|     Municipal Court|37.0|      30.48|                R|              A|
|Hispanic or Latino|  M|   Police Department|46.0|       57.7|                R|              A|
|Hispanic or Latino|  F|Office of Labor S...|null|      46.68|                R|              A|
|Hispanic or Latino|  M|Seattle Dept of T...|26.0|      45.81|                R|              A|
|Hispanic or Latino|  F|Seattle Public Ut...|34.0|      32.48|                R|              A|
|Hispanic or Latino|  M|     Fire Department|47.0|      56.28|                R|              A|
|Hispanic or Latino|  F|     M

In [13]:
sparkDF.printSchema()

root
 |-- Race/Ethnicity: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Hourly Rate: double (nullable = true)
 |-- Regular/Temporary: string (nullable = true)
 |-- Employee Status: string (nullable = true)



In [14]:
df2 = sparkDF.withColumn("Race/Ethnicity",col("Race/Ethnicity").cast('string')) \
    .withColumn("Sex",col("Sex").cast('string')) \
    .withColumn("Department",col("Department").cast('string')) \
    .withColumn('Age', col('Age').cast('integer')) \
    .withColumn('Hourly Rate', col('Hourly Rate').cast('integer')) \
    .withColumn('Regular/Temporary', col('Regular/Temporary').cast('string')) \
    .withColumn('Employee Status', col('Employee Status').cast('string'))

In [16]:
mean_age_row = df2.agg({'Age' : 'avg'}).collect()
mean_age = mean_age_row[0][0]

In [44]:
df2.fillna(value = mean_age, subset = ['Age']).show()

+------------------+---+--------------------+---+-----------+-----------------+---------------+
|    Race/Ethnicity|Sex|          Department|Age|Hourly Rate|Regular/Temporary|Employee Status|
+------------------+---+--------------------+---+-----------+-----------------+---------------+
|Hispanic or Latino|  M|  Parks & Recreation| 42|         36|                R|              A|
|Hispanic or Latino|  F|     Municipal Court| 37|         30|                R|              A|
|Hispanic or Latino|  M|   Police Department| 46|         57|                R|              A|
|Hispanic or Latino|  F|Office of Labor S...| 44|         46|                R|              A|
|Hispanic or Latino|  M|Seattle Dept of T...| 26|         45|                R|              A|
|Hispanic or Latino|  F|Seattle Public Ut...| 34|         32|                R|              A|
|Hispanic or Latino|  M|     Fire Department| 47|         56|                R|              A|
|Hispanic or Latino|  F|     Municipal C